In [1]:
import cv2
import numpy as np
import time 

In [2]:
 # 根据路径读取一张图片
image = cv2.imread("light.jpg")
shapes=image.shape
# 剪切有效区域 ：判断图片的大小，是否需要裁剪
images_size=[shapes[0],shapes[1]]
images_size=np.min(images_size)
if images_size > 1000:
    image=image[0:int(1/3*shapes[1]),int(1/12*shapes[0]):int(11/12*shapes[1]),:]
elif images_size > 800:
    image=image[int(1/6*shapes[1]):int(5/6*shapes[1]),int(1/12*shapes[0]):int(11/12*shapes[1]),:]
else:
    image=image
# 计算有效区域的面积
shapes=image.shape
propotion=image.shape[0]*image.shape[1]
cv2.imshow('image',image)
 # BGR转HSV
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
#保留底片
end=image.copy()

In [3]:
# 绿色的范围
def getGreen(image):    
#     lower_green = np.array([35, 20, 150])
#     upper_green = np.array([70, 255, 255])
#     lower_green = np.array([16, 20, 200])
#     upper_green = np.array([36, 255, 255])
#     lower_green = np.array([60, 40, 150])
#     upper_green = np.array([95, 255, 255])
#     lower_green = np.array([30, 60, 145])
#     upper_green = np.array([50, 255, 255])
    lower_green = np.array([45, 70, 100])
    upper_green = np.array([85, 255, 255])
    return {'green':getRange(image,lower_green,upper_green)}

# 红色的范围
def getRed(image):    
#     low_red = np.array([0, 56, 90])
#     high_red = np.array([9, 203, 255])
#     low_red = np.array([160, 80, 145])
#     high_red = np.array([255, 255, 255])
    low_red = np.array([0, 124, 145])
    high_red = np.array([8, 255, 255])
    return {'red':getRange(image,low_red,high_red)}

# 绿色的范围
def getYellow(image):    
#     low_yellow = np.array([16, 20, 200])
#     high_yellow = np.array([36, 235, 255])
    low_yellow = np.array([15, 100, 125])
    high_yellow = np.array([30, 255, 255])
    return {'yellow':getRange(image,low_yellow,high_yellow)}

def getRange(image,low,high):
# cv2.inRange函数设阈值，去除背景部分
    mask = cv2.inRange(image, low, high)
    cv2.imshow('mask',mask)
    return mask

def deNoise(img):   
#灰度化
#     gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# 腐蚀   
#     cv2.imshow('img',img)
    erode=cv2.erode(img,(100,100),iterations=4)
    cv2.imshow('erode',erode)
    #降噪（模糊处理用来减少瑕疵点）
    blur = cv2.blur(erode, (5,5))    
    cv2.imshow('blur',blur)
# 膨胀
    dilate=cv2.dilate(blur,(100,100),iterations=4)
    cv2.imshow('dilate',dilate)

# 边缘检测    
    canny = cv2.Canny(blur, 150, 240)     
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
# 闭操作    
    closed = cv2.morphologyEx(canny, cv2.MORPH_CLOSE, kernel)
    cv2.imshow('closed',closed)
    return closed

def getPositions(img):
    ret,thresh = cv2.threshold(img,127,255,0)
    tmp,contours,hierarchy=cv2.findContours(thresh,1,2)
    print('contours:',len(contours))        
    area_list=[]  
    order=0
#     有效交通灯的比例    
    if contours :
        for i in range(len(contours)):
            if cv2.contourArea(contours[i])/propotion >0.0005 and cv2.contourArea(contours[i])/propotion < 0.1:
                area_list.append((i,cv2.contourArea(contours[i])))
# 画最小外接矩形
                rect=cv2.minAreaRect(contours[i])
                box=np.int0(cv2.boxPoints(rect))        
# #在原图画出所有ROI区域
                cv2.drawContours(end,[box],-1,(0,0,255),2)  
                
                
# ，用一个最小的矩形，把找到的形状包起来。还有一个带旋转的矩形，面积会更小                
#                 x,y,w,h=cv2.boundingRect(contours[i])
#                 cv2.rectangle(end,(x,y),(x+w,y+h),(0,255,0),2)



# 所有的ROI区域就是按从大到小开始排序的
#         area_list=sorted(area_list,key=lambda x : (x[1]),reverse=True)
        print('area_list',area_list)
# 求所有ROI区域中的中位数    
        order=int(np.ceil((len(area_list))/2))
        print('order:',order)
        if order>0:
            cnt=contours[order] 
            
#按最小面积计算 HSV的坐标    
            x,y,w,h=cv2.boundingRect(cnt)
            cv2.rectangle(end,(x,y),(x+w,y+h),(0,255,0),2)
            y0=int(y-h)
            y1=int(y+h)
            x0=int(x-w)
            x1=int(x+w)
            if y0 <0:
                y0=0
            if y1>shapes[0]:
                y1=shapes[0]
            if x0<0:
                x0=0
            if x1>shapes[1]:
                x1=shapes[1]
        

 # #按最小外接矩阵求HSV ROI区域的坐标
    #             box=np.int0(cv2.boxPoints(rect))
    # #         [[341 217]
    # #  [-27 147]
    # #  [  6 -32]
    # #  [375  37]]
    #             print('box',box)
    #             h1=max([box][0][0][1],[box][0][1][1],[box][0][2][1],[box][0][3][1])
    #             h2=min([box][0][0][1],[box][0][1][1],[box][0][2][1],[box][0][3][1])
    #             l1=max([box][0][0][0],[box][0][1][0],[box][0][2][0],[box][0][3][0])
    #             l2=min([box][0][0][0],[box][0][1][0],[box][0][2][0],[box][0][3][0])
    #             h1 =h1 if h1 > 0 else 0
    #             h2 =h2 if h2 > 0 else 0
    #             l1 =h1 if l1 > 0 else 0
    #             l2 =l2 if l2 > 0 else 0
    #             print('(l1,h1), (l2,h2)',(l1,h1), (l2,h2))
    # #             cv2.rectangle(end, (l1,h1), (l2,h2), (0, 255, 0), 2)    
              
 



 #   #         最小外切圆
    #             (x,y),radius = cv2.minEnclosingCircle(cnt)
    #             center = (int(x),int(y))
    #             radius = int(radius)
        #         cv2.circle(end,center,radius,(0,0,255),2)
        #         cv2.rectangle(end, (int(x-radius),int(y-radius)), (int(x+radius),int(y+radius)), (0, 255, 0), 2)
        #         越界处理
    #             y0=int(y-radius)
    #             y1=int(y+radius)
    #             x0=int(x-radius)
    #             x1=int(x+radius)
    #             if y0 <0:
    #                 y0=0
    #             if y1>shapes[0]:
    #                 y1=shapes[0]
    #             if x0<0:
    #                 x0=0
    #             if x1>shapes[1]:
    #                 x1=shapes[1]

        
                
            cv2.imshow('endne',end)
#             return [h2,h1,l2,l1]
            return [y0,y1,x0,x1]
        else:
            return 0
#         return 1
    else:
        return 0
    


In [4]:
colors=[]
colors.append(getGreen(hsv))
colors.append(getRed(hsv))
colors.append(getYellow(hsv))
# 节省时间复杂度
start=time.clock()
# positions=[]
areas=[]
sumResult=0
_max={'color':0}
for i in range(len(colors)):
    closed = deNoise(list(colors[i].values())[0])
    coordinate = getPositions(closed)
    if isinstance(coordinate,list):
        print('coordinate',coordinate)
#         cv2.imshow('endend',end)
        HSV=end[coordinate[0]:coordinate[1],coordinate[2]:coordinate[3],:]
#         cv2.imshow('HSV',HSV)
        HSV = cv2.cvtColor(HSV, cv2.COLOR_BGR2HSV)
        
        H, S, V = cv2.split(HSV)
        v = np.mean(V)
        print('v',v)
        if v >list( _max.values())[0]:
            _max['color']=v
            _max[list(colors[i].keys())[0]] = _max.pop('color')
        sumResult+=v
        areas.append((list(colors[i].keys())[0],v))
    else:
        areas.append((list(colors[i].keys())[0],0))
        
# 按颜色值的大小排序

areas=sorted(areas,key=lambda x : (x[1]),reverse=True)
print(areas)
print(_max)
ended=time.clock()
print('total:',ended-start)
if int(areas[0][1]) != 0:
    print('The current color is:',areas[0][0])
else:
    print('没有交通灯')
cv2.waitKey(0)
cv2.destroyAllWindows()

contours: 9
area_list [(0, 291.0), (1, 347.5), (2, 1333.0), (3, 229.5), (4, 239.5), (7, 6832.0)]
order: 3
coordinate [496, 534, 197, 229]
v 201.686677632
contours: 14
area_list [(1, 306.5), (2, 901.5), (4, 113.0), (5, 172.0), (6, 118.5), (8, 673.0), (10, 298.5), (11, 18140.5), (12, 311.0), (13, 527.5)]
order: 5
coordinate [240, 272, 87, 131]
v 116.176136364
contours: 5
area_list [(0, 109.0), (1, 160.0), (2, 6025.0), (3, 193.5), (4, 11034.5)]
order: 3
coordinate [287, 321, 155, 187]
v 234.123161765
[('yellow', 234.12316176470588), ('green', 201.68667763157896), ('red', 116.17613636363636)]
{'green': 201.68667763157896, 'yellow': 234.12316176470588}
total: 0.28960191497579335
The current color is: yellow
